## Some nice tools are still in python2

### TF

Calculating forward kinematics is one of most frequent things in robotics.

We want to know how each object (say robot) is oriented relative to some other object (say a door handle)
so we can then calculate how to move one towards another.

Because of that coordinate frames (TF -- transform frames) that describe how elements are related is an important object.

Currentely a set of coordinate frames is published for us by robot_state_publisher. Let's read one such frame


In [ ]:
rospy.init_node('tf_listener')

from tf2_msgs.msg import TFMessage
tf_message=rospy.wait_for_message("/tf",TFMessage)
print(tf_message)

As usuall, there is a lot of info in such a single message. But the important part are two:
one is the child - parent relation between frame_id and child_frame_id. 

/tf topic has messages that are *relations* between some coordinate systems. In can be realition between a ground and a robot but it can be a relation between a robot torso and a robot shoulder. To build a whole robot structure we collect a set of such relations to build a *kinematic tree*.

Let's see how this kinematic tree looks for our robot

In [ ]:
!rosrun tf view_frames && evince frames.pdf

But actually we normally do not use these messagages directely, instead we use tf.TransformListener'ers (instead of Subscribers) and tf.TransformBroadcaster's. This is  because we can select some particular transformations that we need and the listener will calculate them for us. 

For example, let's say we want to know the position of robot's head relative to its base

In [ ]:
import tf
import rospy
## here we just move robot to some base position



listener = tf.TransformListener()


rate = rospy.Rate(10.0)

trans=None

while(trans is None):
    try:

            (trans,rot) = listener.lookupTransform( 'right_shoulder_link','torso_link', rospy.Time(0))

    except (tf.LookupException, tf.ConnectivityException, tf.ExtrapolationException) as e:
        pass
    rate.sleep()
        
        
print("Position",trans)
print("Rotation in quaternions",rot)

    

## Rosbag

Rosbag is a tool for recording the stream of topics and than replaying or analising them

In [ ]:
!rosbag record /tf -O demo.bag

In [ ]:
!rosbag info demo.bag

In [ ]:
import rosbag
from itertools import islice

bag = rosbag.Bag('demo.bag')
for topic, msg, t in islice(bag.read_messages(topics=['/tf']),0,10):
    print msg
bag.close()


## You have finished the practical part of the workshop! CONGRATULATIONS!
Lets wrap up all